In [ ]:
# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# MRI imports
# -----------
import nibabel as nb
import cortex

# Functions import
# ----------------
from utils import draw_cortex_vertex, set_pycortex_config_file

# GLM imports
# ----------------
from nilearn import image, datasets, plotting, surface
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_design_matrix, plot_stat_map, plot_anat, plot_img

In [ ]:
def eventsMatrix(design_file):
    tr_dur = 1.2
    events = pd.read_table(design_file)

    if 'VE' not in task: # Sac/Pur Loc
        events_glm = events[['onset','duration','trial_type']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.trial_type!=events_glm.trial_type.shift()).cumsum())

        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            type = group[1]['trial_type'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,type],index =['onset', 'duration', 'trial_type']), ignore_index=True)

    else: # Visual-Endogenous Sac/Loc    
        events_glm = events[['onset','duration','trial_type', 'eyemov_vis_end']].copy(deep=True)
        events_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
        events_glm.replace({'eyemov_vis_end': {3: 'None', 1: 'Vis', 2: 'End'}},inplace=True)
        events_glm.at[:,'duration'] = tr_dur
        events_glm.at[:,'onset'] = 0
        events_glm_groups = events_glm.groupby((events_glm.eyemov_vis_end!=events_glm.eyemov_vis_end.shift()).cumsum())

        new_events_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type', 'vis_end'])
        for idx, group in enumerate(events_glm_groups):
            onset = group[1]['onset'][group[1].index[0]]
            dur = sum(group[1]['duration'])
            type = group[1]['trial_type'][group[1].index[0]]
            vis_end = group[1]['eyemov_vis_end'][group[1].index[0]]

            new_events_glm = new_events_glm.append(pd.Series([onset,dur,type,vis_end],index =['onset', 'duration', 'trial_type', 'vis_end']), ignore_index=True)

    for idx in new_events_glm.index:
        if idx==0:
            new_events_glm.at[idx, 'onset'] = int(0)
        else:
            new_events_glm.at[idx, 'onset'] = new_events_glm.at[idx-1, 'onset'] + new_events_glm.at[idx-1, 'duration']

    return new_events_glm


In [ ]:
# Get inputs
# ----------
#mount_dir = '/media/scratchms/data/PredictEye' #sys.argv[1]
# mount_dir = '/Users/martinszinte/disks/meso_S/data/PredictEye/'
mount_dir = '/home/mszinte/disks/meso_S/data/PredictEye/'
subject = 'sub-17'#sys.argv[2]
session = 'ses-02'
task = 'SacLoc'
space = 'T1w'
run = 'run-1'
preproc = 'fmriprep_dct'
save_svg = 0
if save_svg == 1: save_svg = True
else: save_svg = False
    
    
# Define analysis parameters
# --------------------------
with open('../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define folder
# -------------
xfm_name = "identity.fmriprep"
base_dir = "{}".format(mount_dir)
deriv_dir = "{}/pp_data/{}/glm/fit".format(base_dir,subject)

task_name = [task[:3], 'Fix' ]


file_img = "{cwd}/pp_data/{subject}/func/{subject}_task-{task}_space-{space}_{preproc}_avg.nii.gz".\
                format(cwd=mount_dir, subject=subject,task=task,space=space,preproc=preproc)

file_mask_img = '{cwd}/deriv_data/fmriprep/fmriprep/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_space-{space}_desc-brain_mask.nii.gz'.\
                format(cwd=mount_dir, subject=subject, session=session, task=task, run=run, space=space)


output_folder = '{cwd}/pp_data/{subject}/glm/fit/'.format(cwd=mount_dir, subject=subject)


try:
    os.makedirs(output_folder)
except:
    pass

# create design table
design_file_run1 = '{cwd}/bids_data/{subject}/{session}/func/{subject}_{session}_task-{task}_{run}_events.tsv'.\
                    format(cwd=mount_dir, subject=subject, session=session, task=task, run='run-01')
events_glm = eventsMatrix(design_file_run1)

In [ ]:
events_glm

In [ ]:
# first level GLM
fmri_img = file_img
output_fn = '{output_folder}{subject}_task-{task}_space-{space}_{preproc}_deriv.nii.gz'.\
            format(output_folder=output_folder, subject=subject,task=task,space=space,preproc=preproc)
mask_img = nb.load(file_mask_img)

fmri_glm = FirstLevelModel(t_r=1.2,
                        noise_model='ar1',
                        standardize=False,
                        hrf_model='spm',
                        drift_model=None,
                        mask_img=mask_img)

fmri_glm = fmri_glm.fit(fmri_img, events_glm)

# design matrix
design_matrix = fmri_glm.design_matrices_[0]
plot_design_matrix(design_matrix)
plt.show()

# contrast
if 'VE' not in task:
    exec('conditions = { task_name[1]: np.array([1., 0., 0.]), task_name[0]: np.array([0., 1., 0.])}')
else:
    exec('conditions = { task_name[1]: np.array([1., 0., 0., 0., 0.]), task_name[0]: np.array([0., 1., 0., 0., 0.]), \
            \'Vis\': np.array([0., 0., 1., 0., 0.]), \'End\': np.array([0., 0., 0., 1., 0.])}')


contrast = conditions[task_name[0]] - conditions[task_name[1]]
# plotting.plot_contrast_matrix(contrast, design_matrix=design_matrix)

eff_map = fmri_glm.compute_contrast(contrast,
                                    output_type='effect_size')

z_map = fmri_glm.compute_contrast(contrast,
                                    output_type='z_score')

p_map = 1 - stats.norm.cdf(abs(z_map.dataobj))

# stats maps
fdr_map, th = threshold_stats_img(z_map, alpha=0.01, height_control='fdr')
fdr_cluster10_map, th = threshold_stats_img(z_map, alpha=0.01, height_control='fdr', cluster_threshold=10)
fdr_cluster50_map, th = threshold_stats_img(z_map, alpha=0.01, height_control='fdr', cluster_threshold=50)
fdr_cluster100_map, th = threshold_stats_img(z_map, alpha=0.01, height_control='fdr', cluster_threshold=100)

# Save results
img = nb.load(file_img)
deriv = np.zeros((img.shape[0],img.shape[1],img.shape[2],6))*np.nan
deriv[...,0]  = z_map.dataobj
deriv[...,1]  = p_map
deriv[...,2]  = fdr_map.dataobj
deriv[...,3]  = fdr_cluster10_map.dataobj
deriv[...,4]  = fdr_cluster50_map.dataobj
deriv[...,5]  = fdr_cluster100_map.dataobj

deriv = deriv.astype(np.float32)
new_img = nb.Nifti1Image(dataobj = deriv, affine = img.affine, header = img.header)
new_img.to_filename(output_fn)

In [ ]:
# Set pycortex db and colormaps
# -----------------------------
set_pycortex_config_file(base_dir)


depth = 1
thick = 1
sampler = 'nearest'
with_curvature = True
with_labels = False
with_colorbar = False
with_borders = False 
curv_brightness = 0.85
curv_contrast = 0.25
add_roi = False


z_map = deriv[...,0].T
p_map = deriv[...,1].T
fdr_map = deriv[...,2].T
fdr_c10_map = deriv[...,3].T
fdr_c50_map = deriv[...,4].T
fdr_c100_map = deriv[...,5].T


In [ ]:
cmap = 'BuWtRd_alpha'

input_map = fdr_c100_map;

volume = cortex.Volume2D(dim1=input_map,
                         dim2=1-p_map,
                         subject=subject,
                         xfmname=xfm_name,
                         description='GLM: Sac vs. Fix',
                         cmap=cmap,
                         vmin=-3.5,
                         vmax=3.5,
                         vmin2=1-0.01,
                         vmax2=1-0.0001
                        )
volume_fig = cortex.quickshow(  braindata = volume,
                                depth = depth,
                                thick = thick,
                                sampler = sampler,
                                with_curvature = with_curvature,
                                with_labels = with_labels,
                                with_colorbar = with_colorbar,
                                with_borders = with_borders,
                                curvature_brightness = curv_brightness,
                                curvature_contrast = curv_contrast)


In [ ]:
# cmap = 'BuWtRd'

# volume = cortex.Volume(data=fdr_map,
#                         subject=subject,
#                          xfmname=xfm_name,
#                          description='GLM: Sac vs. Fix',
#                          cmap=cmap,
#                          vmin=-3.5,
#                          vmax=3.5,
#                         )

# volume_fig = cortex.quickshow(  braindata = volume)

In [ ]:
# cmap = 'BuWtRd'

# volume = cortex.Volume(data=fdr_c10_map,
#                         subject=subject,
#                          xfmname=xfm_name,
#                          description='GLM: Sac vs. Fix',
#                          cmap=cmap,
#                          vmin=-3.5,
#                          vmax=3.5,
#                         )

# volume_fig = cortex.quickshow(  braindata = volume)

In [ ]:
# cmap = 'BuWtRd'

# volume = cortex.Volume(data=fdr_c100_map,
#                         subject=subject,
#                          xfmname=xfm_name,
#                          description='GLM: Sac vs. Fix',
#                          cmap=cmap,
#                          vmin=-3.5,
#                          vmax=3.5,
#                         )

# volume_fig = cortex.quickshow(  braindata = volume)

In [ ]:
# cmap = 'BuWtRd_alpha'

# volume = cortex.Volume2D(dim1=fdr_c50_map,
#                          dim2=1-p_map,
#                          subject=subject,
#                          xfmname=xfm_name,
#                          description='GLM: Sac vs. Fix',
#                          cmap=cmap,
#                          vmin=-3.5,
#                          vmax=3.5,
#                          vmin2=1-0.01,
#                          vmax2=1-0.0001
#                         )
# volume_fig = cortex.quickshow(  braindata = volume,
#                                 depth = depth,
#                                 thick = thick,
#                                 sampler = sampler,
#                                 with_curvature = with_curvature,
#                                 with_labels = with_labels,
#                                 with_colorbar = with_colorbar,
#                                 with_borders = with_borders,
#                                 curvature_brightness = curv_brightness,
#                                 curvature_contrast = curv_contrast)


In [ ]:
# cmap = 'BuWtRd_alpha'

# volume = cortex.Volume2D(dim1=fdr_map,
#                          dim2=1-p_map,
#                          subject=subject,
#                          xfmname=xfm_name,
#                          description='GLM: Sac vs. Fix',
#                          cmap=cmap,
#                          vmin=-3.5,
#                          vmax=3.5,
#                          vmin2=1-0.01,
#                          vmax2=1-0.0001
#                         )
# volume_fig = cortex.quickshow(  braindata = volume,
#                                 depth = depth,
#                                 thick = thick,
#                                 sampler = sampler,
#                                 with_curvature = with_curvature,
#                                 with_labels = with_labels,
#                                 with_colorbar = with_colorbar,
#                                 with_borders = with_borders,
#                                 curvature_brightness = curv_brightness,
#                                 curvature_contrast = curv_contrast)
